In [13]:
import os.path as op
import os
import pandas as pd
from scgc.classifier import BlastHits
from scgc.classifier import BLAST6
from viruscope import run_blast, readfa, run_prodigal
from scgc.utils import file_transaction, run, safe_makedir, file_exists
import yaml

In [14]:
config = yaml.load(open("../vcprofiler.yaml"))

In [15]:
config

{'databases': {'blast': {'djr': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/djr/djr_proteins.fasta',
   'imgvr': {'nucl': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_mVCs_nucleotides.fna',
    'prot': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_mVCs_proteins.faa'},
   'ssdna_virs': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/ssdna/ssDNA_viruses.fna',
   'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta'},
  'hmm': {'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/hmm/vog_hmm'},
  'taxonomy': {'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv'}}}

In [16]:
db = config['databases']['blast']['vog']

In [5]:
?run_blast

In [6]:
?BLAST6

In [17]:
def blastp(fasta, out_file, blast_db, threads=1, numalign=10, evalue=0.001, override=False):
    """
    align sequences using blastp, using -m 6 tab delimited output format.
    fasta : file path as string
    out_file : result file path with tsv extension
    options : blastp options except for db, num_threads, query, out, and outfmt
    returns : outfile path
    """
    fields = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 
              'qend', 'sstart', 'send', 'evalue', 'bitscore', 'score', 'nident', 
              'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qlen', 'slen']
    
    if file_exists(out_file) and not override:
        return out_file
    
    with file_transaction(out_file) as tx_out_file:
        cmd = run_blast(fasta, out_file, db=blast_db, num_alignments=numalign, evalue=evalue, threads=threads, fields=fields)
        
        run(cmd)

    return out_file

def blastn(fasta, out_file, blast_db, threads=1, numalign=10, evalue=0.001, override=False):
    """
    align sequences using blastp, using -m 6 tab delimited output format.
    fasta : file path as string
    out_file : result file path with tsv extension
    options : blastp options except for db, num_threads, query, out, and outfmt
    returns : outfile path
    """
    fields = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 
              'qend', 'sstart', 'send', 'evalue', 'bitscore', 'score', 'nident', 
              'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qlen', 'slen']
    
    if file_exists(out_file) and not override:
        return out_file
    
    with file_transaction(out_file) as tx_out_file:
        cmd = run_blast(fasta, out_file, db=blast_db, num_alignments=numalign, evalue=evalue, threads=threads, fields=fields, blasttype='blastn')
        
        run(cmd)

    return out_file

In [18]:
testdir = safe_makedir("/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/")

In [19]:
test_contigs = "/mnt/scgc/simon/simonsproject/vs_analyses/bats248_vs_analyses/180327_vf_pos.fasta"
totest = op.join(testdir, "bats248_vspos_subset.fasta")

In [20]:
totest

'/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset.fasta'

In [16]:
!grep -c ">" {test_contigs}

9570


In [111]:
count = 0
names = set()
with open(op.join(testdir, "bats248_vspos_subset.fasta"), "w") as oh: 
    for name, seq in readfa(open(test_contigs)):
        names.add("_".join(name.split()[0].split("_")[:-1]))
        if len(names) > 10:
            break
        else:
            count += 1
            print(">{}".format(name), file=oh)
            for i in range(0, len(seq), 60):
                print(seq[i:i+60], file=oh)

In [112]:
!grep -c ">" {totest}

223


In [113]:
names

{'AG-359-N20_NODE_1',
 'AG-359-O04_NODE_20_length_3025_cov_15.5505_ID_39',
 'AG-359-O06_NODE_1',
 'AG-359-O06_NODE_2',
 'AG-359-O06_NODE_3',
 'AG-359-O19_NODE_1',
 'AG-359-O19_NODE_2',
 'AG-359-O19_NODE_3',
 'AG-359-O19_NODE_4',
 'AG-359-O19_NODE_5',
 'AG-359-O19_NODE_6'}

In [114]:
!head {totest}

>AG-359-N20_NODE_1_1 # 2 # 160 # 1 # ID=1_1;partial=10;start_type=Edge;rbs_motif=None;rbs_spacer=None;gc_cont=0.447
DPKTGKMIPKFAMDGKGKMNRGGMMKKKGMAKGGAMMKKKGMAKGGAARRGR*
>AG-359-N20_NODE_1_2 # 160 # 501 # 1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.363
MSTIQTSSALRVARVDLTTTDNTTLYTVPNRYDSIVESIIVSEDSGNADTITITITDAAS
AVFNLFKVKAISANGTVELLSRDLLLTAGDILKVQAATANRLHVVASITEIPS*
>AG-359-N20_NODE_1_3 # 501 # 680 # 1 # ID=1_3;partial=00;start_type=ATG;rbs_motif=AGGA/GGAG/GAGG;rbs_spacer=11-12bp;gc_cont=0.406
MAAKSTVNKAKNYTKPTMRKNLFNKIKAGGKGGNPGQWSARKAQMLAKQYKAKGGGYKS*
>AG-359-N20_NODE_1_4 # 681 # 848 # 1 # ID=1_4;partial=00;start_type=ATG;rbs_motif=AGGAG(G)/GGAGG;rbs_spacer=13-15bp;gc_cont=0.357
MGHYTKRLTKVVGALNKASKAHAGQAKTLKAILKDQKSYHSGKKKRPQSRNRKKA*
>AG-359-N20_NODE_1_5 # 802 # 1095 # 1 # ID=1_5;partial=00;start_type=GTG;rbs_motif=GGA/GAG/AGG;rbs_spacer=5-10bp;gc_cont=0.367


In [115]:
!rm {outblast}

In [21]:
outblast = "{}_vs_{}_blastp.out".format(totest.split(".")[0], 'vog')
blastp(totest, outblast, db, override=True)

blastp -db /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta -query /mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset.fasta -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore score nident positive gaps ppos qframe sframe qlen slen' -num_threads 1 -num_alignments 10 -evalue 0.001 >> /mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset_vs_vog_blastp.out


'/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset_vs_vog_blastp.out'

In [117]:
fields = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 
              'qend', 'sstart', 'send', 'evalue', 'bitscore', 'score', 'nident', 
              'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qlen', 'slen']
bdf = pd.read_csv(outblast, names = fields, sep="\t")

In [33]:
def get_best_hits(blastdf, min_align_pct = 0, min_pct_id = 0, min_bitscore = 0):
    blastdf = blastdf[(blastdf['qlen'] / blastdf['length'] >= min_align_pct / 100) & (blastdf['pident'] >= min_pct_id) &  (blastdf['bitscore'] >= min_bitscore)] 
    return blastdf.sort_values(by=['qseqid','bitscore'], ascending=False).drop_duplicates(subset=['qseqid'], keep='first')

In [120]:
bdf_top = get_best_hits(bdf, min_bitscore=50, min_align_pct = 90)

In [98]:
bdf.columns

Index(['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
       'qend', 'sstart', 'send', 'evalue', 'bitscore', 'score', 'nident',
       'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qlen', 'slen', 'vog',
       'vog_desc'],
      dtype='object')

In [141]:
def summarise_vog_blast(bdf):
    bdf['vog'] = [i.split("__")[-1].split(":")[0] for i in bdf['sseqid']]
    bdf['vog_desc'] = [i.split("__")[-1].split(":")[1].replace("_"," ") for i in bdf['sseqid']]
    bdf['contig'] = ["_".join(i.split("_")[:-1]) for i in bdf['qseqid']]
    outdf = bdf[['contig','qseqid','vog','vog_desc']].rename(columns={'qseqid':'orf'})
    outdf['orf_num'] = [int(i.split("_")[-1]) for i in outdf['orf']]
    outdf = outdf.sort_values(by=['contig','orf_num']).reset_index()
    return outdf[['contig','orf','vog','vog_desc']]

def hits_per_contig(bdf):
    out = pd.DataFrame(summarise_vog_blast(bdf).groupby('contig')['vog'].count()).reset_index()
    return out.rename(columns={'vog':'vog_count'})

In [142]:
vogs_per_contig(bdf_top)

,contig,vog_count
0,AG-359-N20_NODE_1,29
1,AG-359-O06_NODE_1,9
2,AG-359-O06_NODE_2,6
3,AG-359-O06_NODE_3,1
4,AG-359-O19_NODE_1,24
5,AG-359-O19_NODE_2,6
6,AG-359-O19_NODE_3,16
7,AG-359-O19_NODE_4,5
8,AG-359-O19_NODE_5,11


In [173]:
vog_lca = pd.read_csv(config['databases']['taxonomy']['vog'], sep="\t")
vog_lca = vog_lca.rename(columns=dict(zip(vog_lca.columns, ['vog','GenomesInGroupAndLCA', 'GenomesInLCA', 'LCA'])))
vog_lca[:10]

,vog,GenomesInGroupAndLCA,GenomesInLCA,LCA
0,VOG00001,55,2593,"Viruses;dsDNA viruses, no RNA stage"
1,VOG00002,21,2593,"Viruses;dsDNA viruses, no RNA stage"
2,VOG00003,129,2593,"Viruses;dsDNA viruses, no RNA stage"
3,VOG00004,488,2593,"Viruses;dsDNA viruses, no RNA stage"
4,VOG00005,51,2593,"Viruses;dsDNA viruses, no RNA stage"
5,VOG00006,313,5069,Viruses
6,VOG00007,365,5069,Viruses
7,VOG00008,30,2593,"Viruses;dsDNA viruses, no RNA stage"
8,VOG00009,187,2593,"Viruses;dsDNA viruses, no RNA stage"
9,VOG00010,33,2593,"Viruses;dsDNA viruses, no RNA stage"


In [203]:
def vog_lca_per_contig(bdf_top, vog_lca_file):
    vog_lca = pd.read_csv(vog_lca_file, sep="\t")
    vog_lca = vog_lca.rename(columns=dict(zip(vog_lca.columns, ['vog','GenomesInGroupAndLCA', 'GenomesInLCA', 'LCA'])))
    
    best_guesses = []
    vbdf = summarise_vog_blast(bdf_top).merge(vog_lca[['vog','LCA']], on='vog', how='left')

    clca_count = vbdf.groupby(['contig','LCA'])['orf'].count().reset_index().sort_values(by=['contig','orf'], ascending=False).reset_index()
    for c in clca_count['contig'].unique():
        subdf = clca_count[clca_count['contig'] == c]
        maxcount = 0
        for i, l in subdf.iterrows():
            if l['orf'] > maxcount:
                maxcount = l['orf']
                best_guess = l
            elif l['orf'] == maxcount:
                if len(l['LCA'].split(";")) > len(best_guess['LCA'].split(";")):
                    best_guess = l
                else:
                    continue
        best_guesses.append(best_guess)
    return pd.DataFrame(best_guesses)[['contig', 'LCA','orf']].rename(columns={'orf':'lca_hit_count', 'LCA':'vog_lca'})

In [204]:
def vog_per_contig_summary(bdf_top):
    return pd.merge(vogs_per_contig(bdf_top), vog_lca(bdf_top, config['databases']['taxonomy']['vog']), on='contig', how='outer')

,contig,vog_count,vog_lca,hit_count
0,AG-359-N20_NODE_1,29,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",16
1,AG-359-O06_NODE_1,9,"Viruses;dsDNA viruses, no RNA stage;Caudoviral...",5
2,AG-359-O06_NODE_2,6,"Viruses;dsDNA viruses, no RNA stage;Caudoviral...",3
3,AG-359-O06_NODE_3,1,"Viruses;dsDNA viruses, no RNA stage",1
4,AG-359-O19_NODE_1,24,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",19
5,AG-359-O19_NODE_2,6,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",2
6,AG-359-O19_NODE_3,16,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",9
7,AG-359-O19_NODE_4,5,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",4
8,AG-359-O19_NODE_5,11,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",7


Now IMGVR Blast:

In [160]:
db = config['databases']['blast']['imgvr']['prot']
outblast = totest.replace(".fasta","_vs_imgvr_blastp.out")
blastout = blastp(totest, outblast, db, threads=4)

blastp -db /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_mVCs_proteins.faa -query /mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset.fasta -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore score nident positive gaps ppos qframe sframe qlen slen' -num_threads 4 -num_alignments 10 -evalue 0.001 >> /mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset_vs_imgvr_blastp.out


In [19]:
def import_blastp(blastpath, fields=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
      'qend', 'sstart', 'send', 'evalue', 'bitscore', 'score', 'nident', 'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qlen', 'slen']):
    return pd.read_csv(blastpath, names=fields, sep="\t")

In [17]:
blastout = totest.replace(".fasta","_vs_imgvr_blastp.out")

In [20]:
imgbin = import_blastp(blastout)

In [28]:
img_top = get_best_hits(imgbin, min_align_pct=95, min_bitscore=50)

In [29]:
img_top

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,...,bitscore,score,nident,positive,gaps,ppos,qframe,sframe,qlen,slen
1935,AG-359-O19_NODE_5_9,3300006345_____Ga0099693_1028406_____Ga0099693...,82.81,128,21,1,1,127,1,128,...,196.0,497,106,119,1,92.97,1,1,130,135
1925,AG-359-O19_NODE_5_8,3300009550_____Ga0115013_10001311_____Ga011501...,82.31,130,23,0,1,130,1,130,...,225.0,573,107,120,0,92.31,1,1,130,130
1915,AG-359-O19_NODE_5_7,3300006345_____Ga0099693_1028406_____Ga0099693...,77.46,71,16,0,1,71,1,71,...,116.0,291,55,64,0,90.14,1,1,75,73
1905,AG-359-O19_NODE_5_6,3300011013_____Ga0114934_10008206_____Ga011493...,69.23,78,24,0,1,78,1,78,...,119.0,298,54,70,0,89.74,1,1,78,78
1895,AG-359-O19_NODE_5_5,3300009790_____Ga0115012_10001870_____Ga011501...,88.89,54,6,0,1,54,1,54,...,98.2,243,48,53,0,98.15,1,1,54,54
1891,AG-359-O19_NODE_5_4,3300005521_____Ga0066862_10000206_____Ga006686...,56.47,85,37,0,1,85,1,85,...,96.3,238,48,63,0,74.12,1,1,85,85
2129,AG-359-O19_NODE_5_30,3300000117_____DelMOWin2010_c10000241_____DelM...,76.34,93,22,0,1,93,78,170,...,143.0,360,71,82,0,88.17,1,1,93,170
1881,AG-359-O19_NODE_5_3,3300012920_____Ga0160423_10010745_____Ga016042...,78.73,268,47,2,5,267,9,271,...,425.0,1092,211,233,10,86.94,1,1,270,289
2119,AG-359-O19_NODE_5_29,3300012920_____Ga0160423_10002891_____Ga016042...,69.86,73,22,0,1,73,1,73,...,109.0,273,51,59,0,80.82,1,1,73,73
2109,AG-359-O19_NODE_5_28,3300012920_____Ga0160423_10010745_____Ga016042...,78.73,268,47,2,5,267,9,271,...,425.0,1092,211,233,10,86.94,1,1,270,289


In [30]:
img_top['mVCs'] = ["_____".join(i.split("_____")[:-1]) for i in img_top['sseqid']]

In [5]:
imgi = pd.read_csv('/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_Sequence_information.tsv', sep="\t")

/mnt/scgc_nfs/opt/common/anaconda3/4.0.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (10,11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
imgi.columns

Index(['mVCs', 'TAXON_OID', 'Scaffold_ID', 'VIRAL_CLUSTERS', 'Ecosystem',
       'Ecosystem_Category', 'Ecosystem_Type', 'Ecosystem_Subtype', 'Habitat',
       'perc_VPF', 'Host', 'Host_detection', 'POGs_ORDER', 'POGs_FAMILY',
       'POGs_SUBFAMILY', 'POGs_GENUS', 'putative_retrovirus'],
      dtype='object')

In [32]:
imgt1 = img_top.merge(imgi, on='mVCs', how='left')

In [37]:
imgt1

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,...,Habitat,perc_VPF,Host,Host_detection,POGs_ORDER,POGs_FAMILY,POGs_SUBFAMILY,POGs_GENUS,putative_retrovirus,contig
0,AG-359-O19_NODE_5_9,3300006345_____Ga0099693_1028406_____Ga0099693...,82.81,128,21,1,1,127,1,128,...,Marine,0.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
1,AG-359-O19_NODE_5_8,3300009550_____Ga0115013_10001311_____Ga011501...,82.31,130,23,0,1,130,1,130,...,Marine,0.423077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
2,AG-359-O19_NODE_5_7,3300006345_____Ga0099693_1028406_____Ga0099693...,77.46,71,16,0,1,71,1,71,...,Marine,0.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
3,AG-359-O19_NODE_5_6,3300011013_____Ga0114934_10008206_____Ga011493...,69.23,78,24,0,1,78,1,78,...,Marine,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
4,AG-359-O19_NODE_5_5,3300009790_____Ga0115012_10001870_____Ga011501...,88.89,54,6,0,1,54,1,54,...,Marine,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
5,AG-359-O19_NODE_5_4,3300005521_____Ga0066862_10000206_____Ga006686...,56.47,85,37,0,1,85,1,85,...,Marine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
6,AG-359-O19_NODE_5_30,3300000117_____DelMOWin2010_c10000241_____DelM...,76.34,93,22,0,1,93,78,170,...,Marine,0.517241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
7,AG-359-O19_NODE_5_3,3300012920_____Ga0160423_10010745_____Ga016042...,78.73,268,47,2,5,267,9,271,...,Marine,0.411765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
8,AG-359-O19_NODE_5_29,3300012920_____Ga0160423_10002891_____Ga016042...,69.86,73,22,0,1,73,1,73,...,Marine,0.580645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5
9,AG-359-O19_NODE_5_28,3300012920_____Ga0160423_10010745_____Ga016042...,78.73,268,47,2,5,267,9,271,...,Marine,0.411765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AG-359-O19_NODE_5


In [41]:
imgi['TAXON_OID']

0         3300011994
1         3300014037
2         3300011985
3         3300011980
4         3300011981
5         3300011988
6         3300014025
7         3300011976
8         3300011976
9         3300014025
10        3300011974
11        3300011974
12        3300011972
13        3300014591
14        3300011948
15        3300011936
16        3300011981
17        3300011990
18        3300011931
19        3300012275
20        3300011926
21        3300011915
22        3300012281
23        3300011894
24        3300013170
25        3300011883
26        3300011875
27        3300011864
28        3300011905
29        3300011863
             ...    
715642    3300006991
715643    3300006979
715644    3300006979
715645    3300006979
715646    3300014059
715647    3300006979
715648    3300006877
715649    3300006406
715650    3300006238
715651    3300006238
715652    3300006200
715653    3300014040
715654    3300014040
715655    3300014040
715656    3300014040
715657    3300014059
715658    330

In [50]:
def hits_per_contig(bdf, hits_category):
    bdf['contig'] = ["_".join(i.split("_")[:-1]) for i in bdf['qseqid']]
    out = bdf.groupby('contig')['qseqid'].count().reset_index()
    out = out.rename(columns={'qseqid':'orfs_matching_{}'.format(hits_category)})
    return out


In [51]:
hits_per_contig(imgt1, "imgvr")

,contig,orfs_matching_imgvr
0,AG-359-N20_NODE_1,61
1,AG-359-O04_NODE_20_length_3025_cov_15.5505_ID_39,2
2,AG-359-O06_NODE_1,15
3,AG-359-O06_NODE_2,8
4,AG-359-O06_NODE_3,6
5,AG-359-O19_NODE_1,32
6,AG-359-O19_NODE_2,9
7,AG-359-O19_NODE_3,36
8,AG-359-O19_NODE_4,16
9,AG-359-O19_NODE_5,29


In [53]:
from collections import defaultdict
contig_orf_count = defaultdict(lambda:0)

In [57]:
for name, seq in readfa(open(totest)):
    contig_orf_count["_".join(name.split()[0].split("_")[:-1])] += 1

In [64]:

pd.DataFrame(list(contig_orf_count.items()), columns=['contig','total_orfs'])

,contig,total_orfs
0,AG-359-N20_NODE_1,66
1,AG-359-O19_NODE_4,17
2,AG-359-O19_NODE_1,33
3,AG-359-O04_NODE_20_length_3025_cov_15.5505_ID_39,2
4,AG-359-O19_NODE_5,30
5,AG-359-O06_NODE_2,8
6,AG-359-O06_NODE_1,15
7,AG-359-O06_NODE_3,6
8,AG-359-O19_NODE_3,37
9,AG-359-O19_NODE_2,9


Single Stranded DNA viruses:

In [11]:
blastdb = config['databases']['blast']['ssdna_virs']

In [69]:
blastdb

'/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/ssdna/ssDNA_viruses.fna'

In [9]:
infa = '/mnt/scgc/simon/simonsproject/vs_analyses/bats248_vs_analyses/180418_recip_sized.fasta'
outfile = '/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/recip_vir_lens_vs_ssDNAvirs_blastn.out'

In [12]:
res = blastn(infa, outfile, blastdb, threads=4, numalign=10, evalue=0.001, override=False)

blastn -db /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/ssdna/ssDNA_viruses.fna -query /mnt/scgc/simon/simonsproject/vs_analyses/bats248_vs_analyses/180418_recip_sized.fasta -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore score nident positive gaps ppos qframe sframe qlen slen' -num_threads 4 -num_alignments 10 -evalue 0.001 >> /mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/recip_vir_lens_vs_ssDNAvirs_blastn.out


In [13]:
res

'/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/recip_vir_lens_vs_ssDNAvirs_blastn.out'

In [16]:
# that worked, but there are no matches to the test set!
# added a djr_proteins database

In [18]:
?run_prodigal

In [19]:
prodout = safe_makedir('../test_data/small_contigs_prod')

In [20]:
prots1 = run_prodigal(infa, prodout)

Running Prodigal on /mnt/scgc/simon/simonsproject/vs_analyses/bats248_vs_analyses/180418_recip_sized.fasta


In [21]:
!grep -c ">" {prots1}

395


In [6]:
blastdb = config['databases']['blast']['djr']
blastdb
outblast = "../test_data/180418_contig_orfs_vs_djr.out"

In [30]:
outblast = blastp(prots1, outblast, blastdb, threads=4)

blastp -db /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/djr/djr_proteins.fasta -query ../test_data/small_contigs_prod/180418_proteins.fasta -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore score nident positive gaps ppos qframe sframe qlen slen' -num_threads 4 -num_alignments 10 -evalue 0.001 >> ../test_data/180418_contig_orfs_vs_djr.out


In [31]:
outblast

'../test_data/180418_contig_orfs_vs_djr.out'

In [5]:
from vcprofiler import import_blastp, get_best_hits

In [10]:
bdf = get_best_hits(import_blastp(outblast), min_align_pct=75, min_pct_id=35, min_bitscore=50)

In [12]:
bdf

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,...,bitscore,score,nident,positive,gaps,ppos,qframe,sframe,qlen,slen
